In [45]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import yfinance as yf
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import pandas_ta as ta
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [46]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [47]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    df = pd.DataFrame(ohlc)
    df['time'] = pd.to_datetime(df['time'], unit= 's')
    return df

In [98]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,6,1)
end_datetime = datetime.now()
df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2025-06-02 00:00:00,1.13463,1.13463,1.13398,1.13398,2,70,0
1,2025-06-02 00:01:00,1.13398,1.13398,1.13398,1.13398,1,200,0
2,2025-06-02 00:02:00,1.13398,1.13398,1.13398,1.13398,1,200,0
3,2025-06-02 00:03:00,1.13398,1.13398,1.13390,1.13390,2,200,0
4,2025-06-02 00:04:00,1.13390,1.13390,1.13382,1.13382,2,200,0
...,...,...,...,...,...,...,...,...
43916,2025-07-14 12:40:00,1.16881,1.16881,1.16851,1.16857,57,0,0
43917,2025-07-14 12:41:00,1.16857,1.16870,1.16856,1.16864,35,0,0
43918,2025-07-14 12:42:00,1.16864,1.16880,1.16858,1.16880,46,0,0
43919,2025-07-14 12:43:00,1.16880,1.16886,1.16877,1.16879,40,0,0


In [49]:
spread_threshold = 5  # in points
atr_period = 14

# Calculate metrics
#df['spread'] = (df['ask'] - df['bid']) * 10000  # Convert to pips
df['vwap'] = (df['close'] * df['tick_volume']).cumsum() / df['tick_volume'].cumsum()
df['atr'] = df[['high', 'low', 'close']].apply(
    lambda row: max(row['high'] - row['low'],
                    abs(row['high'] - row['close']),
                    abs(row['low'] - row['close'])),
    axis=1
)
df['atr'] = df['atr'].rolling(atr_period).mean()

# Strategy condition
df['volume_percentile'] = df['tick_volume'].rolling(atr_period).apply(
    lambda x: np.percentile(x, 75), raw=True
)
df

,time,open,high,low,close,tick_volume,spread,real_volume,vwap,atr,volume_percentile
0,2025-06-02 00:00:00,1.13463,1.13463,1.13398,1.13398,2,70,0,1.133980,NaN,NaN
1,2025-06-02 00:01:00,1.13398,1.13398,1.13398,1.13398,1,200,0,1.133980,NaN,NaN
2,2025-06-02 00:02:00,1.13398,1.13398,1.13398,1.13398,1,200,0,1.133980,NaN,NaN
3,2025-06-02 00:03:00,1.13398,1.13398,1.13390,1.13390,2,200,0,1.133953,NaN,NaN
4,2025-06-02 00:04:00,1.13390,1.13390,1.13382,1.13382,2,200,0,1.133920,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
43657,2025-07-14 08:21:00,1.16764,1.16771,1.16764,1.16767,20,0,0,1.158590,0.000107,33.75
43658,2025-07-14 08:22:00,1.16766,1.16771,1.16757,1.16760,23,0,0,1.158591,0.000108,33.75
43659,2025-07-14 08:23:00,1.16760,1.16760,1.16742,1.16743,25,0,0,1.158591,0.000114,32.75
43660,2025-07-14 08:24:00,1.16743,1.16746,1.16730,1.16730,32,0,0,1.158591,0.000122,33.50


In [50]:
df['high'] = df['high'].shift(1)

In [56]:
def entring_stoploss_profit(row, df) :
    if (row['spread'] < spread_threshold) & (row['close'] > row['vwap']) &  (row['tick_volume'] > row['volume_percentile']) &(row['close'] > row['high']):
        return "buy"
    #elif :
        #return "sell"
df["signal"] = df.apply(entring_stoploss_profit, axis=1, df = df)

# Tag first signal per day in original merged_df
#df['is_first_signal'] = (df[df["signal"].notna()].groupby(df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
#df['signal'] = df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
df[['signal', 'time']].dropna()

,signal,time
63,buy,2025-06-02 01:03:00
78,buy,2025-06-02 01:18:00
79,buy,2025-06-02 01:19:00
86,buy,2025-06-02 01:26:00
122,buy,2025-06-02 02:02:00
...,...,...
43611,buy,2025-07-14 07:35:00
43615,buy,2025-07-14 07:39:00
43616,buy,2025-07-14 07:40:00
43640,buy,2025-07-14 08:04:00


In [95]:
pd.options.mode.copy_on_write = True
# create trade logic
def on_bar(data, trades, orders):
    data['stop_loss'] = data['close'] - data['atr']
    data['take_profit'] = data['close'] + 2 * data['atr']
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy': #and not num_open_trades:
        volume = 100000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell': #and not num_open_trades:
        volume = 100000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and (data['high'] >= data['take_profit']) | (data['low'] <= data['stop_loss']):
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' :
            orders.close_trade(trade)

In [96]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,100000,2025-06-02 01:03:00,1.13512,2025-06-02 01:05:00,1.1352,0,0,{},8.0,-7.0,1.0,1.0,10001.0
1,closed,EURUSD,buy,100000,2025-06-02 01:18:00,1.13548,2025-06-02 01:26:00,1.13517,0,0,{},-31.0,-7.0,-38.0,-37.0,9963.0
2,closed,EURUSD,buy,100000,2025-06-02 01:19:00,1.13565,2025-06-02 01:44:00,1.13485,0,0,{},-80.0,-7.0,-87.0,-124.0,9876.0
3,closed,EURUSD,buy,100000,2025-06-02 01:26:00,1.13517,2025-06-02 01:46:00,1.13504,0,0,{},-13.0,-7.0,-20.0,-144.0,9856.0
4,closed,EURUSD,buy,100000,2025-06-02 02:02:00,1.1355,2025-06-02 02:21:00,1.13561,0,0,{},11.0,-7.0,4.0,-140.0,9860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,closed,EURUSD,buy,100000,2025-07-14 07:35:00,1.16775,2025-07-14 07:39:00,1.16791,0,0,{},16.0,-7.0,9.0,758.0,10758.0
2996,closed,EURUSD,buy,100000,2025-07-14 07:39:00,1.16791,2025-07-14 07:44:00,1.16791,0,0,{},0.0,-7.0,-7.0,751.0,10751.0
2997,closed,EURUSD,buy,100000,2025-07-14 07:40:00,1.16805,2025-07-14 07:45:00,1.16781,0,0,{},-24.0,-7.0,-31.0,720.0,10720.0
2998,closed,EURUSD,buy,100000,2025-07-14 08:04:00,1.16724,2025-07-14 08:05:00,1.16745,0,0,{},21.0,-7.0,14.0,734.0,10734.0


In [70]:
mm = bt.plot_balance()
display(mm)

In [60]:
evaluate_backtest(bt.trades)

biggest_profit: 342.0
daily_drawdown: -341.0
max_drawown: -2815.0
max_drawown1: -255.0
avg_win: 24.39
avg_loss: -27.43
count_profit_trades: 1983
count_loss_trades: 971
Win Rate1: 67.13%
rrr: 0.89


,order_type,profit
0,buy,21723.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit_net,profit_cumulative,balance,current_max,drawdown,drawdown1,dayofweek,hourofday,month,year
0,closed,EURUSD,buy,100000,2025-06-02 01:03:00,1.13512,2025-06-02 01:05:00,1.1352,0,0,...,1.0,1.0,10001.0,1.0,0.0,-255.0,0,1,2025-06,2025
1,closed,EURUSD,buy,100000,2025-06-02 01:18:00,1.13548,2025-06-02 01:26:00,1.13517,0,0,...,-38.0,-37.0,9963.0,1.0,-38.0,-255.0,0,1,2025-06,2025
2,closed,EURUSD,buy,100000,2025-06-02 01:19:00,1.13565,2025-06-02 01:44:00,1.13485,0,0,...,-87.0,-124.0,9876.0,1.0,-125.0,-255.0,0,1,2025-06,2025
3,closed,EURUSD,buy,100000,2025-06-02 01:26:00,1.13517,2025-06-02 01:46:00,1.13504,0,0,...,-20.0,-144.0,9856.0,1.0,-145.0,-255.0,0,1,2025-06,2025
4,closed,EURUSD,buy,100000,2025-06-02 02:02:00,1.1355,2025-06-02 02:21:00,1.13561,0,0,...,4.0,-140.0,9860.0,1.0,-141.0,-255.0,0,2,2025-06,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,closed,EURUSD,buy,100000,2025-07-14 07:35:00,1.16775,2025-07-14 07:39:00,1.16791,0,0,...,9.0,758.0,10758.0,3237.0,-2479.0,-255.0,0,7,2025-07,2025
2996,closed,EURUSD,buy,100000,2025-07-14 07:39:00,1.16791,2025-07-14 07:44:00,1.16791,0,0,...,-7.0,751.0,10751.0,3237.0,-2486.0,-255.0,0,7,2025-07,2025
2997,closed,EURUSD,buy,100000,2025-07-14 07:40:00,1.16805,2025-07-14 07:45:00,1.16781,0,0,...,-31.0,720.0,10720.0,3237.0,-2517.0,-255.0,0,7,2025-07,2025
2998,closed,EURUSD,buy,100000,2025-07-14 08:04:00,1.16724,2025-07-14 08:05:00,1.16745,0,0,...,14.0,734.0,10734.0,3237.0,-2503.0,-255.0,0,8,2025-07,2025


ValueError: Plotly Express cannot process wide-form data with columns of different type.